In [1]:
######### 始める前に設定必要の変数：###########

#### 1. BT : Academic申請後に生成される Bearer Token(BT)
#### 2. TWEET_LIMIT_you_set : 一回のクエリーの上限
#### 3. queryword_path : 辞書のエクセルファイル
#### 4. savedir_you_set : csvの保存フォルダー

#BT = '################'

#TWEET_LIMIT_you_set = 100000                   ####  テストするときには1000とか設定した方がいいです
                                            #### 毎月の上限MAXは 10,000,000 なので　9,900,000にすることがおすすめです

savedir_you_set = './'

In [2]:
import requests
import os
import json
import time
import pandas as pd


import datetime
from pytz import timezone


### look up tweet by id
### documentation : https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets-id


search_url = 'https://api.twitter.com/2/tweets/'


#与えられたパラメータからツイート取得条件(query_params)を生成する関数
def make_parm(id_string):


    query_params = {'ids': id_string,
                    'tweet.fields': 'created_at,conversation_id',
                    'expansions': 'author_id,geo.place_id',
                    'user.fields': 'created_at,description,location,username,protected,verified',
                    'place.fields': 'contained_within,country,country_code,full_name,geo'
#                     'max_results': 500,  #一回のqueryは５００で上限らしい
#                     'next_token' : {} #次のページにいくためのparam
                   }
    return query_params


def create_headers(BT):
    headers = {"Authorization": "Bearer {}".format(BT)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        #raise Exception(response.status_code, response.text)
    
        print(response.status_code, response.text)
        
    return response   ######## to make use of  header information, return response instead of response.json()

    
def json_to_df(json_response):
    
    normalized_data = pd.json_normalize(json_response['data'])
    normalized_users = pd.json_normalize(json_response['includes']['users']).set_index('id')
     
    normalized_users.rename({'id': 'author_id'}, inplace = True, axis = 1)
    normalized_data.rename({'geo.place_id': 'place_id'}, inplace = True, axis = 1)
            
    normalized = normalized_data.join(normalized_users, on = 'author_id', how = 'outer', rsuffix = '_user')
    
    if 'places' in json_response['includes'].keys():
        normalized_places = pd.json_normalize(json_response['includes']['places']).set_index('id')
        normalized_places.rename({'id': 'place_id'}, inplace = True, axis = 1)
        normalized = normalized.join(normalized_places, on = 'place_id', how = 'outer', rsuffix = '_place')
    
    normalized['text'] = normalized['text'].str.replace('\n', ' ')     #text内の\nを削除
    normalized['text'] = normalized['text'].str.replace('@', ' @')     #@前にスペースを追加
    
    return normalized
    
def main(id_string):

    query_params = make_parm(id_string)#ここで取得条件が返される
    
    headers = create_headers(BT)
    time.sleep(1)
        
    response = connect_to_endpoint(search_url, headers, query_params)
        
    if response.status_code != 200:
        print(str(response.status_code) + 'Error occurred.')
        
    json_response = response.json()  
        
        
    print(json_response['data']) 
    #print(json_response) 
    normalized_data = json_to_df(json_response)
    #import pdb;pdb.set_trace()
    return normalized_data['conversation_id'][0],normalized_data["text"]



In [3]:
##### ＧｏＴｏ再開へ準備のツイート

tweet_id = '1502525843968790528'
conversation_id,text = main(id_string = tweet_id)

[{'id': '1502525843968790528', 'created_at': '2022-03-12T06:04:11.000Z', 'text': 'ＧｏＴｏ再開へ準備―岸田首相\n\nなんでそうなるのか。特定の業種のみに税金を投入するやり方は有効ではない。結局は利権。市中の無料検査体制と医療提供体制を整え、社会規制を緩和できる社会づくりを目指すべき。なぜ2年あってやれないのか。また、中途半端の繰り返しになる。 https://t.co/lL1bfmmCFP', 'author_id': '188620283', 'conversation_id': '1502525843968790528'}]


In [4]:
##### このconversation_idを使って会話に参加したユーザーの発言を収集する
##### キーワード検索と同様に、ただ具体的な単語ではなく、queryを'conversation_id:XXXXXXXXXXXX'にする

search_url = "https://api.twitter.com/2/tweets/search/all"

#与えられたパラメータからツイート取得条件(query_params)を生成する関数
def make_parm_kw(kw):

    query_params = {'query': kw,
                    'tweet.fields': 'created_at,conversation_id',
                    'expansions': 'author_id,geo.place_id',
                    'user.fields': 'created_at,description,location,username,protected,verified',
                    'place.fields': 'contained_within,country,country_code,full_name,geo'
#                     'max_results': 500,  #一回のqueryは５００で上限らしい
#                     'next_token' : {} #次のページにいくためのparam
                   }
    return query_params



def main_kw(kw):

    query_params = make_parm_kw(kw)
    
    headers = create_headers(BT)
    time.sleep(1)
        
    response = connect_to_endpoint(search_url, headers, query_params)
        
    if response.status_code != 200:
        print(str(response.status_code) + 'Error occurred.')
        
    json_response = response.json()  
    normalized_data = json_to_df(json_response)
    
    return normalized_data

### tweet_idから引用RTを取得

In [5]:
main_kw("1502525843968790528 -is:retweet")

,conversation_id,author_id,created_at,text,id,name,created_at_user,description,verified,protected,username,location
0,1505791409944932354,477363475,2022-03-21T06:20:23.000Z,自民の血。 https://t.co/4pTKTgpOHv,1505791409944932354,相良博之@ジャーナル,2012-01-29T03:41:24.000Z,向こうも見ずに独立後あれよあれよと振り返れば30年。想定外の貧しき現役法人役員。子供が全員巣...,False,False,sgr_press,NaN
1,1505317197438394369,268048924,2022-03-19T22:56:01.000Z,GO TOは必要ないと思う。旅行したい人は勝手にする。 私はコロナ感染を避けたいので旅行はし...,1505317197438394369,rrwwnn,2011-03-18T01:52:51.000Z,,False,False,rrwwnn,NaN
2,1504400209723670528,1224708219886612480,2022-03-17T10:12:15.000Z,#GoTo再開 #小沢一郎 「特定業種」「利権」「社会規制」「中途半端」 当たり前に叫ばれる...,1504400209723670528,꧁✨𝕛𝔭 -𝒴𝓊𝒩𝒾-📚 右派左派、政党と その思想に関わらず。,2020-02-04T14:56:20.000Z,"†hê mðrê Ì lêårñ, †hê mðrê Ì rêålïzê Ì Ððñ’† k...",False,False,ENlyrdfq5gaHetG,NaN
3,1504400165683478535,100140955,2022-03-17T10:12:04.000Z,「GoTo敵基地先制攻撃だ」「GoTo戦術核兵器共有だ」 https://t.co/pgu1...,1504400165683478535,知足隠遁,2009-12-29T05:08:34.000Z,自動制御技術者⇒知足的生活 ・・・歪曲偏向指摘、隣国報道紹介＝脱原発。脱官僚専制。脱軍国主...,False,False,SukiyakiSong,三重県松阪市か千葉県柏市
4,1504382605357359107,1417726326136397824,2022-03-17T09:02:17.000Z,何時の頃からか賭場と遊山が本邦の産業の中心のように扱われ始めた。産業競争力の低下もあるが政治...,1504382605357359107,田舎者がきらい 「れいわ」に小口寄付しよう,2021-07-21T06:01:36.000Z,政治難民、猫の思い出とワインの日々。\nこのアカウントのテーマは官僚諸悪の根源論です。\n保...,False,False,pwzFmZ1zMhDYiAk,東京
5,1504271628888588289,1219755292541382656,2022-03-17T01:41:18.000Z,GoTo再開って…この国大丈夫？ ため息しか出てこない。 #岸田政権は早期に総辞職せよ #...,1504271628888588289,Yoshi Kubo,2020-01-21T22:55:24.000Z,アメリカ育ちの🇯🇵人\n日本の政治は変わらないと近未来大変なことになるでしょう\n少しでも良...,False,False,yoshikubo7,"Shinagawa-ku, Tokyo"
6,1504217408336064512,542437486,2022-03-16T22:05:51.000Z,効果の無い施策をなぜ続けるのか https://t.co/Tue5DcciNR,1504217408336064512,flomx,2012-04-01T10:26:11.000Z,"I started reading books, one book/month at a t...",False,False,flomx1,NaN
7,1504214767149658112,1369816676,2022-03-16T21:55:22.000Z,自民党は頭が少し足りないのです。自分達のことしか結局考えてない。自民党以外に任せられる政党が...,1504214767149658112,773,2013-04-21T15:21:23.000Z,#投票倍増委員会\n#赤木さんを忘れない\nおかしいことはおかしいと声を上げ続けます。,False,False,yori_kw,NaN
8,1504095662551760901,247205070,2022-03-16T14:02:05.000Z,gotoやるなら、pcr検査を いつでも・どこでも・無料で・何回でも とセットでなければ感染...,1504095662551760901,5656,2011-02-04T09:26:55.000Z,#札幌五輪反対 #新潮社不買 #大阪都構想詐欺 #大阪万博反対 #脱原発 #マスク #リニア...,False,False,nyannnya4,JPN
9,1504061339706269699,1087034469230567424,2022-03-16T11:45:42.000Z,特定業種のみに税金を投入するのはなぜ？ それを追求するのがメディアと野党の役割のはずなんで...,1504061339706269699,ゐ(うぃ),2019-01-20T17:09:48.000Z,🇳🇱在住、アラ還の駐妻、海外出羽守。欧州調査歴30年以上、検索魔。園芸歴40年以上。ミニチュ...,False,False,aromi317oakleaf,"Amsterdam, the Netherlands"
